#事前準備
ライブラリをインストールします。

In [ ]:
# Install Vertex AI LLM SDK
! pip install google-cloud-aiplatform google-cloud-discoveryengine google-auth --upgrade --user

ランタイムを再起動します。自動で再起動された場合はスキップしてください。

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

`Vertex AI User`権限のあるアカウントでログインします。

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

Google Cloud の Vertex AI を利用できるように設定します。

In [ ]:
import vertexai
PROJECT_ID = "<your_project_id>"  # @param {type:"string"}
REGION = "asia-northeast1"

vertexai.init(project=PROJECT_ID, location=REGION)

#Gemini APIの応答
はじめに Gemini に直接質問を投げてみます。

In [ ]:
prompt = """
Cloud Spannerの特徴的な機能は？
"""

In [ ]:
from IPython.display import Markdown
from vertexai.generative_models import GenerativeModel

model = GenerativeModel("gemini-1.5-pro")
responses = model.generate_content([prompt], generation_config={"max_output_tokens": 2048, "temperature": 1.0}, stream=False)
Markdown(responses.text)
# streaming ver
#for response in responses:
#    print(response.text, end="")


#Vertex AI Search の確認
1. Cloud Console の UI で Vertex AI Search の検索アプリを作成してください。

  (UI) https://console.cloud.google.com/gen-app-builder/engines

  (ガイド) https://cloud.google.com/generative-ai-app-builder/docs/try-enterprise-search#create_and_preview_a_search_app_for_structured_data_from


2. 検索対象の文書を複数登録してください。

3. 作成後の検索アプリのプレビュー機能で、検索アプリが正しく動作することを確認します。

4. 作成後のDATASTORE_IDを確認して変更してください。

In [ ]:
LOCATION = "global"
DATASTORE_ID = "<your_datastore_id>" # @param {type:"string"}
DATA_STORE_MODE = "extractive_answers" # @param ["extractive_answers", "extractive_segments", "CHUNKS"]

API でも検索結果が取得できることを確認します。

In [ ]:
from typing import List
from google.cloud import discoveryengine_v1alpha as discoveryengine
from google.api_core.client_options import ClientOptions

search_query = prompt

client = discoveryengine.SearchServiceClient(
    client_options=ClientOptions(api_endpoint=f"{LOCATION}-discoveryengine.googleapis.com")
)

# e.g. projects/{project_id}/locations/{location}/dataStores/{data_store_id}/servingConfigs/{serving_config_id}
serving_config=client.serving_config_path(
    project=PROJECT_ID,
    location=LOCATION,
    data_store=DATASTORE_ID,
    serving_config="default_search:search",
)
query_expansion_spec=discoveryengine.SearchRequest.QueryExpansionSpec(
    condition=discoveryengine.SearchRequest.QueryExpansionSpec.Condition.AUTO,
)
spell_correction_spec=discoveryengine.SearchRequest.SpellCorrectionSpec(
    mode=discoveryengine.SearchRequest.SpellCorrectionSpec.Mode.AUTO
)

# `SearchRequest` https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1alpha.types.SearchRequest
if DATA_STORE_MODE == "CHUNKS":
    page_size=10
    # `ContentSearchSpec` https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1alpha.types.SearchRequest.ContentSearchSpec
    content_search_spec = discoveryengine.SearchRequest.ContentSearchSpec(
        search_result_mode="CHUNKS",#"CHUNKS"
    )
else:
    page_size=3
    # `ContentSearchSpec` https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1alpha.types.SearchRequest.ContentSearchSpec
    content_search_spec = discoveryengine.SearchRequest.ContentSearchSpec(
        # https://cloud.google.com/generative-ai-app-builder/docs/snippets
        snippet_spec=discoveryengine.SearchRequest.ContentSearchSpec.SnippetSpec(
            return_snippet=True
        ),
        # https://cloud.google.com/generative-ai-app-builder/docs/get-search-summaries
        summary_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec(
            summary_result_count=3,
            include_citations=True,
            ignore_adversarial_query=True,
            ignore_non_summary_seeking_query=True,
            # https://cloud.google.com/generative-ai-app-builder/docs/get-search-summaries#summary-model
            model_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec.ModelSpec(
                version="stable"
            )
        ),
        # https://cloud.google.com/generative-ai-app-builder/docs/snippets#extractive-segments
        extractive_content_spec=discoveryengine.SearchRequest.ContentSearchSpec.ExtractiveContentSpec(
            max_extractive_answer_count=3,
            max_extractive_segment_count=3,
            return_extractive_segment_score=True,
        ),
    )

request = discoveryengine.SearchRequest(
    serving_config=serving_config,
    query_expansion_spec=query_expansion_spec,
    spell_correction_spec=spell_correction_spec,
    query=search_query,
    page_size=page_size,
    content_search_spec=content_search_spec,
)

vais_response = client.search(request)


Vertex AI Search の検索サマリーを確認します。CHUNKS モードの場合はありません。

In [ ]:
Markdown(vais_response.summary.summary_text)

参考までに API の応答を全部確認します。

In [ ]:
vais_response

# Vertex AI Search で DIY RAG を実装

DIY でプロンプトを定義し、DIY で Geimini を利用した RAG アプリを作成します。

In [ ]:
context = "<context>\n"
for result in vais_response.results:
    if DATA_STORE_MODE == "CHUNKS":
        chunk = result.chunk
        context += "  <chunk>" + chunk.content
        context += "\n    <reference>" + chunk.document_metadata.uri + "    </reference>\n"
        context += "  </chunk>\n"
    else:
        doc_data = result.document.derived_struct_data
        context += "  <document>\n"
        for chunk in doc_data.get(DATA_STORE_MODE, []):
            context += "    <chunk>" + chunk.get("content", "")
            context += "\n      <referencePage>" + doc_data.get("link","") + ": " +chunk.get("pageNumber", "") + "    </referencePage>\n"
            context += "    </chunk>\n"
            context += "  </document>"

context += "</context>\n"

rag_prompt = """
あなたは与えられたContextを元に回答するRAGエージェントです。Context情報を元に最後の質問に回答してください。
Contextに回答に必要な情報がない場合は、適切な情報が見つかりませんでした、と回答してください。
また、回答には引用した文書とpageNumberをすべて提示してください。
{context}
=====
Question: {prompt}
Ansewer:
"""

rag_prompt = rag_prompt.format(context=context, prompt=prompt)
print(rag_prompt)

In [ ]:
responses = model.generate_content([rag_prompt], generation_config={"max_output_tokens": 2048, "temperature": 0.5}, stream=False)

Markdown(responses.text)

# Grounding with Vertex AI Search

Vertex AI Search でグラウンディングした Gemini の応答を確認します。いわゆる Google マネージドな RAG です。

DATASTOREがCHUNKSモードの場合は利用できません。

In [ ]:
from vertexai.preview.generative_models import GenerativeModel, grounding, Tool

model = GenerativeModel("gemini-1.5-pro-preview-0409")
data_store_path = f"projects/{PROJECT_ID}/locations/{LOCATION}/collections/default_collection/dataStores/{DATASTORE_ID}"
tool = Tool.from_retrieval(grounding.Retrieval(grounding.VertexAISearch(datastore=data_store_path)))
responses = model.generate_content([prompt], generation_config={"max_output_tokens": 2048, "temperature": 0.5}, stream=False, tools=[tool])

Markdown(responses.text)